# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
#import ipywidgets as widgets
# Import API key
from api_keys import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv = "weather_data.csv"
weather_df = pd.read_csv(weather_csv)
weather_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Pangnirtung,CA,66.1451,-65.7125,44.60,76,75,12.66
1,Kutum,SD,14.2000,24.6667,80.37,39,92,7.07
2,Nagato,JP,34.3833,131.2000,71.51,67,91,7.38
3,Bundaberg,AU,-24.8500,152.3500,56.21,87,4,1.99
4,Millinocket,US,45.6573,-68.7098,81.36,50,20,9.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Drop na values
weather_df = weather_df.dropna()

# Humidity
humidity = weather_df["Humidity"]

# Lat and Lng 
locations = weather_df[["Latitude", "Longitude"]]

# Create the map
fig = gmaps.figure()

# Create the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
weather_df = weather_df.dropna()
weather_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Pangnirtung,CA,66.1451,-65.7125,44.60,76,75,12.66
1,Kutum,SD,14.2000,24.6667,80.37,39,92,7.07
2,Nagato,JP,34.3833,131.2000,71.51,67,91,7.38
3,Bundaberg,AU,-24.8500,152.3500,56.21,87,4,1.99
4,Millinocket,US,45.6573,-68.7098,81.36,50,20,9.22


In [9]:
narrowed_weather_df = weather_df.loc[(weather_df["Temperature"] > 70) & (weather_df["Temperature"] < 80) & (weather_df["Cloudiness"] == 0)]
narrowed_weather_df.reset_index(inplace=True)
narrowed_weather_df.head()

,index,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,119,Cape Town,ZA,-33.9258,18.4232,73.11,30,0,5.75
1,233,Nikolskoye,RU,59.7035,30.7861,78.69,68,0,3.44
2,246,Horadiz,AZ,39.4488,47.3353,75.97,57,0,5.68
3,306,Boleč,RS,44.7247,20.6033,76.62,22,0,4.12
4,468,Rosetta,EG,31.4044,30.4164,75.02,73,0,4.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Pangnirtung.
Closest hotel is Kilabuk Lodge.
------------
Retrieving Results for Index 1: Kutum.
Closest hotel is Dibbo areaمنطقة دبو.
------------
Retrieving Results for Index 2: Nagato.
Closest hotel is いそう庵.
------------
Retrieving Results for Index 3: Bundaberg.
Closest hotel is Takalvan Motel.
------------
Retrieving Results for Index 4: Millinocket.
Closest hotel is Katahdin Inn & Suites.
------------
Retrieving Results for Index 5: New Norfolk.
Closest hotel is The Shingles Riverside Cottages.
------------
Retrieving Results for Index 6: Major Isidoro.
Closest hotel is Pousada Brilho da Lua.
------------
Retrieving Results for Index 7: Hermanus.
Closest hotel is Misty Waves Boutique Hotel.
------------
Retrieving Results for Index 8: Busselton.
Closest hotel is Observatory Guest House.
------------
Retrieving Results for Index 9: Kenora.
Closest hotel is Maynard Lake Lodge.
------------
Retrieving Results for Index 10: Neijiang.
Closest hotel is H

Closest hotel is Shandrani Beachcomber Resort & Spa.
------------
Retrieving Results for Index 90: Kaiyuan.
Closest hotel is Dongqi Hotel.
------------
Retrieving Results for Index 91: Leamington.
Closest hotel is Seacliffe Inn (The).
------------
Retrieving Results for Index 92: Kirillov.
Closest hotel is Korona.
------------
Retrieving Results for Index 93: Jamestown.
Closest hotel is Hampton Inn & Suites Jamestown.
------------
Retrieving Results for Index 94: Avarua.
Closest hotel is Paradise Inn.
------------
Retrieving Results for Index 95: Norrköping.
Closest hotel is The Lamp Hotel.
------------
Retrieving Results for Index 96: Ulaanbaatar.
Closest hotel is The Blue Sky Hotel and Tower.
------------
Retrieving Results for Index 97: Auki.
Closest hotel is Auki Motel.
------------
Retrieving Results for Index 98: Vestmannaeyjar.
Closest hotel is Glamping & Camping.
------------
Retrieving Results for Index 99: Acapulco de Juárez.
Closest hotel is HS Hotsson Smart Acapulco.
------

Closest hotel is King Eider Inn.
------------
Retrieving Results for Index 176: Río Gallegos.
Closest hotel is Hotel Santa Cruz.
------------
Retrieving Results for Index 177: Denpasar.
Closest hotel is ASTON Denpasar Hotel & Convention Center.
------------
Retrieving Results for Index 178: Tasiilaq.
Closest hotel is The Red House.
------------
Retrieving Results for Index 179: Luang Prabang.
Closest hotel is Villa Oasis.
------------
Retrieving Results for Index 180: Norman Wells.
Closest hotel is Heritage Hotel.
------------
Retrieving Results for Index 181: Datong.
Closest hotel is Holiday Inn Datong City Centre.
------------
Retrieving Results for Index 182: Yar-Sale.
Closest hotel is Yalemd.
------------
Retrieving Results for Index 183: Katsuura.
Closest hotel is Katsuura Hotel Mikazuki.
------------
Retrieving Results for Index 184: Mapastepec.
Closest hotel is Hotel San Carlos.
------------
Retrieving Results for Index 185: Cullman.
Closest hotel is Best Western Fairwinds Inn.


Closest hotel is Apartment on Kievskaya 29.
------------
Retrieving Results for Index 263: Santa Marinella.
Closest hotel is VILLA GREGORACI - b & b, guest houses, Santa Marinella.
------------
Retrieving Results for Index 264: Kamenka.
Missing field/result... skipping.
------------
Retrieving Results for Index 265: Ust-Maya.
Missing field/result... skipping.
------------
Retrieving Results for Index 266: Jonesboro.
Closest hotel is Hilton Garden Inn Jonesboro.
------------
Retrieving Results for Index 267: Klyuchi.
Closest hotel is Vesolaya Mayovka.
------------
Retrieving Results for Index 268: Codrington.
Closest hotel is Codrington Gardens Bed And Breakfast.
------------
Retrieving Results for Index 269: Bambous Virieux.
Closest hotel is Casa Tia Villa.
------------
Retrieving Results for Index 270: Salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH.
------------
Retrieving Results for Index 271: Kruisfontein.
Closest hotel is Oyster Bay House Rental.
------------
Retrieving Resu

Closest hotel is Pousada e Hotel Litoral Sul.
------------
Retrieving Results for Index 349: Chokwé.
Closest hotel is Complexo Sonho Real.
------------
Retrieving Results for Index 350: Toropets.
Closest hotel is Toropa.
------------
Retrieving Results for Index 351: Palāsa.
Closest hotel is Balaji tifen & meals veg & non veg hotel.
------------
Retrieving Results for Index 352: Massaguet.
Closest hotel is Alwatane.
------------
Retrieving Results for Index 353: Diré.
Missing field/result... skipping.
------------
Retrieving Results for Index 354: Evensk.
Missing field/result... skipping.
------------
Retrieving Results for Index 355: Belmonte.
Closest hotel is Pousada Monte Carmelo O Pão.
------------
Retrieving Results for Index 356: Bosaso.
Closest hotel is Red Sea Hotel.
------------
Retrieving Results for Index 357: Half Moon Bay.
Closest hotel is The Ritz-Carlton, Half Moon Bay.
------------
Retrieving Results for Index 358: Kruglolesskoye.
Missing field/result... skipping.
-----

Closest hotel is InTown Suites Extended Stay Denver CO - Englewood.
------------
Retrieving Results for Index 437: Mihăileni.
Missing field/result... skipping.
------------
Retrieving Results for Index 438: Najrān.
Closest hotel is محمد هادي ال فطيح.
------------
Retrieving Results for Index 439: Olafsvik.
Closest hotel is Hotel Olafsvik.
------------
Retrieving Results for Index 440: Strezhevoy.
Closest hotel is Sibiryachka.
------------
Retrieving Results for Index 441: Russell.
Closest hotel is Macs.
------------
Retrieving Results for Index 442: Kibaya.
Closest hotel is Lusaka Hotel.
------------
Retrieving Results for Index 443: Petauke.
Closest hotel is Newline Lodge.
------------
Retrieving Results for Index 444: San Miguel de Tucumán.
Closest hotel is HOTEL MIAMI.
------------
Retrieving Results for Index 445: Paita.
Closest hotel is Hotel Náutico de Paita.
------------
Retrieving Results for Index 446: Raja.
Missing field/result... skipping.
------------
Retrieving Results for

Closest hotel is Baghdad Hotel.
------------
Retrieving Results for Index 523: Petrozavodsk.
Closest hotel is Первый рыбный ресторан "Фрегат".
------------
Retrieving Results for Index 524: Lebanon.
Closest hotel is Hampton Inn & Suites Lebanon.
------------
Retrieving Results for Index 525: Ambikāpur.
Closest hotel is STARiHOTEL Ambikapur.
------------
Retrieving Results for Index 526: Takoradze.
Closest hotel is Raybow International Hotel.
------------
Retrieving Results for Index 527: Nantucket.
Closest hotel is Jared Coffin House.
------------
Retrieving Results for Index 528: Luga.
Closest hotel is U Lysoy Gory Hotel.
------------
Retrieving Results for Index 529: Oistins.
Closest hotel is Butterfly Beach Hotel.
------------
Retrieving Results for Index 530: Ganzhou.
Closest hotel is Laotou Farm Restaurant Hostel.
------------
Retrieving Results for Index 531: Fort Smith.
Closest hotel is Hampton Inn Fort Smith.
------------
Retrieving Results for Index 532: Camalú.
Closest hotel 

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [24]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…